In [1]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd

In [2]:
# load data
df=pd.read_csv('PageBlocks_withoutdupl_02_v02.csv')  

del df['id']
del df['Unnamed: 0']

df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,outlier
0,5.0,7.0,35.0,1.400,0.400,0.657,2.33,14.0,23.0,6.0,0
1,6.0,7.0,42.0,1.167,0.429,0.881,3.60,18.0,37.0,5.0,0
2,6.0,18.0,108.0,3.000,0.287,0.741,4.43,31.0,80.0,7.0,0
3,5.0,7.0,35.0,1.400,0.371,0.743,4.33,13.0,26.0,3.0,0
4,6.0,3.0,18.0,0.500,0.500,0.944,2.25,9.0,17.0,4.0,0


In [3]:
df.shape

(4982, 11)

In [4]:
# data to values
df = df.values

In [5]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/PageBlocks.json")
gc = GCForest(config)

In [6]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,10]
X = df[:,0:10]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

/home/kader/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# number of class
len(np.unique(y_train))

2

In [8]:
          # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-05-15 10:22:54,344][cascade_classifier.fit_transform] X_groups_train.shape=[(3337, 10)],y_train.shape=(3337,),X_groups_test.shape=[(1645, 10)],y_test.shape=(1645,)
[ 2018-05-15 10:22:54,349][cascade_classifier.fit_transform] group_dims=[10]
[ 2018-05-15 10:22:54,357][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-05-15 10:22:54,359][cascade_classifier.fit_transform] group_ends=[10]
[ 2018-05-15 10:22:54,369][cascade_classifier.fit_transform] X_train.shape=(3337, 10),X_test.shape=(1645, 10)
[ 2018-05-15 10:22:54,371][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(3337, 10), X_cur_test.shape=(1645, 10)
[ 2018-05-15 10:22:55,613][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.70%
[ 2018-05-15 10:22:56,633][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.81%
[ 2018-05-15 10:22:57,714][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-05-15 10:23:05,979][cascade_classifier.transform] X_groups_test.shape=[(1645, 10)]
[ 2018-05-15 10:23:05,980][cascade_classifier.transform] group_dims=[10]
[ 2018-05-15 10:23:05,981][cascade_classifier.transform] X_test.shape=(1645, 10)
[ 2018-05-15 10:23:05,983][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1645, 10)


Test Accuracy of ExtraTrees = 98.662614 %


In [10]:
# Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[1609    1]
 [  21   14]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99      1610
        1.0       0.93      0.40      0.56        35

avg / total       0.99      0.99      0.98      1645



In [11]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-05-15 10:23:07,923][cascade_classifier.transform] X_groups_test.shape=[(1645, 10)]
[ 2018-05-15 10:23:07,925][cascade_classifier.transform] group_dims=[10]
[ 2018-05-15 10:23:07,936][cascade_classifier.transform] X_test.shape=(1645, 10)
[ 2018-05-15 10:23:07,938][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1645, 10)


In [12]:
# New data
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-05-15 10:23:09,344][cascade_classifier.fit_transform] X_groups_train.shape=[(3337, 12)],y_train.shape=(3337,),X_groups_test.shape=[(1645, 12)],y_test.shape=(1645,)
[ 2018-05-15 10:23:09,355][cascade_classifier.fit_transform] group_dims=[12]
[ 2018-05-15 10:23:09,360][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-05-15 10:23:09,361][cascade_classifier.fit_transform] group_ends=[12]
[ 2018-05-15 10:23:09,364][cascade_classifier.fit_transform] X_train.shape=(3337, 12),X_test.shape=(1645, 12)
[ 2018-05-15 10:23:09,365][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(3337, 12), X_cur_test.shape=(1645, 12)
[ 2018-05-15 10:23:10,310][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.70%
[ 2018-05-15 10:23:11,525][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.10%
[ 2018-05-15 10:23:12,468][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [13]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-05-15 10:23:19,922][cascade_classifier.transform] X_groups_test.shape=[(1645, 12)]
[ 2018-05-15 10:23:19,934][cascade_classifier.transform] group_dims=[12]
[ 2018-05-15 10:23:19,936][cascade_classifier.transform] X_test.shape=(1645, 12)
[ 2018-05-15 10:23:19,938][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1645, 12)


Test Accuracy of ExtraTrees = 99.088146 %


In [14]:
# Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[1608    2]
 [  13   22]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      1610
        1.0       0.92      0.63      0.75        35

avg / total       0.99      0.99      0.99      1645



In [15]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')
    # Matrix de confusion
    from sklearn.metrics import classification_report, confusion_matrix
    print(confusion_matrix(y_test,y_pred))
    print('\n')
    print(classification_report(y_test,y_pred))

In [16]:
# Deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()

[ 2018-05-15 10:23:21,925][cascade_classifier.fit_transform] X_groups_train.shape=[(3337, 12)],y_train.shape=(3337,),X_groups_test.shape=[(1645, 12)],y_test.shape=(1645,)
[ 2018-05-15 10:23:21,927][cascade_classifier.fit_transform] group_dims=[12]
[ 2018-05-15 10:23:21,935][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-05-15 10:23:21,937][cascade_classifier.fit_transform] group_ends=[12]
[ 2018-05-15 10:23:21,940][cascade_classifier.fit_transform] X_train.shape=(3337, 12),X_test.shape=(1645, 12)
[ 2018-05-15 10:23:21,944][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(3337, 12), X_cur_test.shape=(1645, 12)
[ 2018-05-15 10:23:23,129][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.70%
[ 2018-05-15 10:23:23,911][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.10%
[ 2018-05-15 10:23:25,025][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

('Test Accuracy of ExtraTrees = 99.027356 %', 0, 'fois')
[[1608    2]
 [  14   21]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      1610
        1.0       0.91      0.60      0.72        35

avg / total       0.99      0.99      0.99      1645



[ 2018-05-15 10:23:35,787][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.40%
[ 2018-05-15 10:23:36,528][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.81%
[ 2018-05-15 10:23:37,890][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.91%
[ 2018-05-15 10:23:39,036][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=99.10%
[ 2018-05-15 10:23:39,937][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.70%
[ 2018-05-15 10:23:41,147][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.50%
[ 2018-05-15 10:23:42,229][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.80%
[ 2018-05-15 10:23:43,257][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.80%


('Test Accuracy of ExtraTrees = 99.148936 %', 1, 'fois')
[[1608    2]
 [  12   23]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      1610
        1.0       0.92      0.66      0.77        35

avg / total       0.99      0.99      0.99      1645



[ 2018-05-15 10:23:47,751][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.70%
[ 2018-05-15 10:23:49,061][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.81%
[ 2018-05-15 10:23:50,102][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.91%
[ 2018-05-15 10:23:51,219][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.80%
[ 2018-05-15 10:23:52,427][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.70%
[ 2018-05-15 10:23:53,174][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.80%
[ 2018-05-15 10:23:54,373][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.50%
[ 2018-05-15 10:23:55,597][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.80%


...

('Test Accuracy of ExtraTrees = 98.723404 %', 17, 'fois')
[[1607    3]
 [  18   17]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99      1610
        1.0       0.85      0.49      0.62        35

avg / total       0.99      0.99      0.99      1645



[ 2018-05-15 10:27:02,148][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.40%
[ 2018-05-15 10:27:02,910][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.81%
[ 2018-05-15 10:27:04,006][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.21%
[ 2018-05-15 10:27:05,365][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.80%
[ 2018-05-15 10:27:06,228][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.40%
[ 2018-05-15 10:27:07,305][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.80%
[ 2018-05-15 10:27:08,531][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.50%
[ 2018-05-15 10:27:09,435][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.10%


('Test Accuracy of ExtraTrees = 98.784195 %', 18, 'fois')
[[1608    2]
 [  18   17]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99      1610
        1.0       0.89      0.49      0.63        35

avg / total       0.99      0.99      0.99      1645



[ 2018-05-15 10:27:14,894][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.40%
[ 2018-05-15 10:27:15,678][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.81%
[ 2018-05-15 10:27:16,914][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.91%
[ 2018-05-15 10:27:18,172][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.20%
[ 2018-05-15 10:27:18,918][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.10%
[ 2018-05-15 10:27:20,040][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.20%
[ 2018-05-15 10:27:21,153][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.80%
[ 2018-05-15 10:27:21,891][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.40%


('Test Accuracy of ExtraTrees = 98.784195 %', 19, 'fois')
[[1609    1]
 [  19   16]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99      1610
        1.0       0.94      0.46      0.62        35

avg / total       0.99      0.99      0.99      1645



# ('Test Accuracy of ExtraTrees = 99.148936 %', 1, 'fois')